In [ ]:
pip install split_folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import os
from PIL import Image
import pathlib
import csv
import keras
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianNoise
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import metrics
import warnings
import csv
import tensorflow as tf
import time
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import splitfolders


splitfolders.ratio('/content/drive/MyDrive/breath_training_image', output="./breath_data", seed=1337, ratio=(.8, .2)) # default values

BreathTrainDataGen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255, 
        shear_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True) 

BreathTestDataGen = ImageDataGenerator(rescale=1./255)

#data set loading
BreathTestSet = BreathTrainDataGen.flow_from_directory(
        '/content/drive/MyDrive/breath_training_image',
        target_size=(576, 576),
        batch_size= 16,
        class_mode='categorical',
        shuffle = False)

BreathTrainingSet = BreathTestDataGen.flow_from_directory(
        '/content/drive/MyDrive/breath_training_image',
        target_size= (576, 576),
        batch_size= 16,
        class_mode='categorical',
        shuffle = False)

#model construct
model = Sequential()
input_shape=(576, 576, 3)
#1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))
#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.7))
#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))
#Output layer
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

epochs = 100
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])


model.fit_generator(
        BreathTrainingSet,
        steps_per_epoch= 1,
        epochs= 100,
        validation_data=BreathTrainingSet,
        validation_steps=1)
model.evaluate_generator(generator=BreathTestSet, steps=1)


Copying files: 110 files [00:00, 416.17 files/s]


Found 110 images belonging to 2 classes.
Found 110 images belonging to 2 classes.
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 287, 287, 32)      896       
_________________________________________________________________
average_pooling2d_5 (Average (None, 143, 143, 32)      0         
_________________________________________________________________
activation_16 (Activation)   (None, 143, 143, 32)      0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 654368)            0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 654368)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                41879616  
______________________________________

[0.6604533195495605, 0.6875]

In [ ]:
BreathTestSet.reset()
PredGen = model.predict_generator(BreathTestSet, steps=7, verbose=1)
predicted_class_indices=np.argmax(PredGen,axis=1)

BreathLabels = (BreathTrainingSet.class_indices)
BreathLabels = dict((v,k) for k,v in BreathLabels.items())
predictions = [BreathLabels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames=BreathTestSet.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(BreathLabels)
print(results)
TestAcc=[]
for index, row in results.iterrows():
  if row['Predictions'] in row['Filename']:
    TestAcc.append(1)
  else:
    TestAcc.append(0)

print('test accuracy:'+str(np.mean(TestAcc)))

7/7 [==============================] - 8s 1s/step
{0: 'negative', 1: 'positive'}
                             Filename Predictions
0         negative/breathing-dee2.png    negative
1        negative/breathing-dee20.png    negative
2       negative/breathing-dee300.png    negative
3         negative/breathing-deep.png    negative
4       negative/breathing-deep10.png    negative
..                                ...         ...
105   positive/breathing-shallow1.png    positive
106  positive/breathing-shallow13.png    negative
107   positive/breathing-shallow3.png    positive
108   positive/breathing-shallow4.png    positive
109   positive/breathing-shallow6.png    negative

[110 rows x 2 columns]
test accuracy:0.7
